In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scipy
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=100, dpi_save=600)
    
import scvelo as scv
scv.settings.verbosity = 3
scv.settings.presenter_view = True
scv.logging.print_versions()

import cellrank as cr
cr.settings.verbosity = 3
cr.logging.print_versions()

import matplotlib.pyplot as pl
from matplotlib import rcParams

import os

In [ ]:
WT = sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_MSO/MSO_matrix/WT/WT_raw_after_Scrublet.h5ad')
WT

In [ ]:
KP = sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_MSO/MSO_matrix/KP/KP_raw_after_Scrublet.h5ad')
KP

In [ ]:
EKP = sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_MSO/MSO_matrix/EKP/EKP_raw_after_Scrublet.h5ad')
EKP

In [ ]:
adata = WT.concatenate(KP, EKP, batch_categories=['WT', 'KP', 'EKP'])
adata

In [ ]:
adata.write('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1_raw.h5ad', compression='gzip')

In [ ]:
adata=sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1_raw.h5ad')

In [ ]:
adata=sc.read('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1.h5ad', compression='gzip')

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)
sc.pp.filter_cells(adata, min_genes=50)
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')
adata.var['rpl'] = adata.var_names.str.startswith('Rpl')
adata.var['rps'] = adata.var_names.str.startswith('Rps')
adata

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','rpl','rps'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, keys=['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_rpl','pct_counts_rps'], jitter=0.4, multi_panel=True)
adata

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_rpl')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_rps')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 8000, :]
adata = adata[adata.obs.pct_counts_mt < 30, :]
adata = adata[adata.obs.pct_counts_rpl < 30, :]
adata = adata[adata.obs.pct_counts_rps < 25, :]
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
sc.pl.highly_variable_genes(adata)
print(sum(adata.var.highly_variable))
adata

In [ ]:
adata.raw=adata

In [ ]:
adata = adata[:, adata.var.highly_variable]
adata

In [ ]:
sc.pp.regress_out(adata, keys=['total_counts', 'pct_counts_mt','pct_counts_rpl','pct_counts_rps'], n_jobs=8)
sc.pp.scale(adata, max_value=10)
adata

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sce.pp.harmony_integrate(adata, key='batch', basis='X_pca', adjusted_basis='X_pca_harmony')
adata.obsm['X_pca']=adata.obsm['X_pca_harmony']
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50, knn=True)
sc.tl.leiden(adata, resolution=0.5)
sc.tl.umap(adata)
adata




In [ ]:
sc.pl.umap(adata, color=['batch'], legend_loc='right margin', frameon=False, title='', use_raw=True,
           palette={'WT': 'blue', 'KP': 'orange', 'EKP': 'green'})
sc.pl.umap(adata, color=['leiden'], legend_loc='right margin', frameon=False, title='', use_raw=True, palette='Accent')

In [ ]:
adata

In [ ]:
WT=adata[adata.obs['batch']=='WT']
KP=adata[adata.obs['batch']=='KP']
EKP=adata[adata.obs['batch']=='EKP']

In [ ]:
WT

In [ ]:
KP

In [ ]:
EKP

In [ ]:
sc.pl.dotplot(adata, var_names=['Krt7', 'Krt20', 'Cdx2','Muc1', 'Muc2', 'Muc5ac','Sox2', 'Sox4'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0)

In [ ]:
# check regulon expression level
sc.pl.dotplot(adata, var_names=['Pole4', 'Ezh2', 'E2f3','Gtf2b', 'Gabpa', 'Tfdp1','Sox4', 'E2f7','E2f1','E2f8','Brca1','Gtf2f1'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0, save='regulon')

In [ ]:
sc.pl.umap(adata, color=['Epcam'], legend_loc='right margin', frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='Epcam_featureplot')

In [ ]:
sc.pl.umap(WT, color=['Top2a'], legend_loc='right margin', vmin=0, vmax=3.3,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='WT_Top2a_featureplot')
sc.pl.umap(KP, color=['Top2a'], legend_loc='right margin', vmin=0, vmax=3.3,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='KP_Top2a_featureplot')
sc.pl.umap(EKP, color=['Top2a'], legend_loc='right margin', vmin=0, vmax=3.3,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='EKP_Top2a_featureplot')
sc.pl.umap(WT, color=['Pclaf'], legend_loc='right margin', vmin=0, vmax=2.8,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='WT_Pclaf_featureplot')
sc.pl.umap(KP, color=['Pclaf'], legend_loc='right margin', vmin=0, vmax=2.8,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='KP_Pclaf_featureplot')
sc.pl.umap(EKP, color=['Pclaf'], legend_loc='right margin', vmin=0, vmax=2.8,frameon=False, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='EKP_Pclaf_featureplot')

In [ ]:

sc.pl.dotplot(adata, var_names=['Snhg4','Muc1','Rex1bd','Pglyrp1','Lypd2',
'Bnip3','Eif1','Pgk1',
'Prxl2a','Aqp5','Hspb1',
'Mki67','Top2a','Pclaf',
'Gm42418','Gm26917','Neat1',
'Fxyd3','Mt2','Crip1',
'Platr31','B020031M17Rik','Dcc'], groupby=['leiden'], use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0, save='annotation')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['leiden'], legend_loc='right margin', frameon=False, title='WT', use_raw=True, save='WT_UMAP.pdf')
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['leiden'], legend_loc='right margin', frameon=False, title='KP', use_raw=True, save='KP_UMAP.pdf')
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['leiden'], legend_loc='right margin', frameon=False, title='EKP', use_raw=True, save='EKP_UMAP.pdf')

In [ ]:
adata[adata.obs['batch']=='WT'].obs['leiden'].value_counts()

In [ ]:
adata[adata.obs['batch']=='KP'].obs['leiden'].value_counts()

In [ ]:
adata[adata.obs['batch']=='EKP'].obs['leiden'].value_counts()

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden', method='wilcoxon', use_raw=False)

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
  {group + '_' + key[:3]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(5000).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1_leiden_markers.csv")

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata[adata.obs['batch']=='EKP'], show_gene_labels=False,n_genes=50, groupby='leiden', use_raw=False, swap_axes=False, vmin=-2, vmax=2, cmap='bwr', dendrogram=False,save='heatmap_EKP')

In [ ]:
#for p value
sc.tl.rank_genes_groups(adata, groupby='batch', groups=['KP'], reference='WT', method='wilcoxon', use_raw=True)
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
  {group + '_' + key[:3]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(32285).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/KPvsWT genes.csv")


In [ ]:
#for p value
sc.tl.rank_genes_groups(adata, groupby='batch', groups=['EKP'], reference='WT', method='wilcoxon', use_raw=True)
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
  {group + '_' + key[:3]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(32285).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/EKPvsWT genes.csv")


In [ ]:
#for p value
sc.tl.rank_genes_groups(adata, groupby='batch', groups=['EKP'], reference='KP', method='wilcoxon', use_raw=True)
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
  {group + '_' + key[:3]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(32285).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/EKPvsKP genes.csv")


In [ ]:
#root=adata[adata.obs['leiden'].isin(['2'])]
#root
 
## use this for GSEA whole gene
#sc.tl.rank_genes_groups(root, groupby='batch', groups=['KO'], reference='WT', method='wilcoxon', use_raw=True)
#result = root.uns['rank_genes_groups']
#groups = result['names'].dtype.names
#pd.DataFrame(
#  {group + '_' + key[:3]: result[key][group]
#    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(15685).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_MSO/MSO_matrix/WTvsKOforGSEA.csv")
 
## use this for Drug prediction top 5000 gene
#sc.tl.rank_genes_groups(root, groupby='batch', groups=['KO'], reference='WT', method='wilcoxon', use_raw=False)
#result = root.uns['rank_genes_groups']
#groups = result['names'].dtype.names
#pd.DataFrame(
#  {group + '_' + key[:3]: result[key][group]
#    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(5000).to_csv("/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_MSO/MSO_matrix/WTvsKOforDrug.csv")

In [ ]:
#sc.tl.rank_genes_groups(adata, groupby='batch', groups=['KO'], reference='WT', method='wilcoxon', use_raw=True)

In [ ]:
#result = adata.uns['rank_genes_groups']
#groups = result['names'].dtype.names
#pd.DataFrame(
    #{group + '_' + key[:3]: result[key][group]
    #for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(5000).to_csv("C:/Users/Park_Lab/Documents/KOvsWTforGSEA.csv")

In [ ]:
#sc.tl.rank_genes_groups(adata, groupby='batch', groups=['CKP'], reference='KP', method='wilcoxon', use_raw=True)

In [ ]:
#result = adata.uns['rank_genes_groups']
#groups = result['names'].dtype.names
#pd.DataFrame(
    #{group + '_' + key[:3]: result[key][group]
    #for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).head(5000).to_csv("C:/Users/Park_Lab/Documents/CKPvsKPforGSEA.csv")

In [ ]:
sc.pl.dotplot(adata, var_names=['Mki67','Hopx', 'Lgr5', 'Lrig1', 'Sox4', 'Prom1', 'Aqp5'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0, save='stemness_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['Mki67','Hopx', 'Lgr5', 'Lrig1', 'Sox4', 'Prom1', 'Aqp5'],
           legend_loc='right margin', frameon=False, vmin=0, vmax=4, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='WT_stemness_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['Mki67','Hopx', 'Lgr5', 'Lrig1', 'Sox4', 'Prom1', 'Aqp5'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=4, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='KP_stemness_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['Mki67','Hopx', 'Lgr5', 'Lrig1', 'Sox4', 'Prom1', 'Aqp5'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=4, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='EKP_stemness_marker')

In [ ]:
sc.pl.dotplot(adata, var_names=['Muc1', 'Muc2', 'Muc4', 'Muc5ac', 'Muc5b', 'Muc13'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0, save='muc_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['Muc1', 'Muc2', 'Muc4', 'Muc5ac', 'Muc5b', 'Muc13'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=3, add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='WT_featureplot_Muc')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['Muc1', 'Muc2', 'Muc4', 'Muc5ac', 'Muc5b', 'Muc13'],
           legend_loc='right margin', frameon=False, vmin=0, vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='KP_featureplot_Muc')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['Muc1', 'Muc2', 'Muc4', 'Muc5ac', 'Muc5b', 'Muc13'],
           legend_loc='right margin', frameon=False, vmin=0, vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples', save='EKP_featureplot_Muc')

In [ ]:
sc.pl.dotplot(adata, var_names=['Krt4', 'Krt8', 'Krt10', 'Krt15', 'Krt16', 'Krt19'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0,save='pan_krt_ae1_ae3')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['Krt4', 'Krt8', 'Krt10', 'Krt15', 'Krt16', 'Krt19'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=4,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='WT_pan_krt_ae1_ae3')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['Krt4', 'Krt8', 'Krt10', 'Krt15', 'Krt16', 'Krt19'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=4,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='KP_pan_krt_ae1_ae3')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['Krt4', 'Krt8', 'Krt10', 'Krt15', 'Krt16', 'Krt19'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=4,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='EKP_pan_krt_ae1_ae3')

In [ ]:
#p16 CDKN2A
#Trp53
#CK AE1/3 include CK1, CK3, CK4, CK5, CK6, CK8, CK10, CK14, CK15, CK16, CK19
sc.pl.dotplot(adata, var_names=['Cdkn2a', 'Trp53','Cdh1'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0,save='p16_p53_cdh1')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['Cdkn2a', 'Trp53','Cdh1'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='WT_p16_p53_cdh1')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['Cdkn2a', 'Trp53','Cdh1'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples'save='KP_p16_p53_cdh1')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['Cdkn2a', 'Trp53','Cdh1'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='EKP_p16_p53_cdh1')

In [ ]:
#Gastric Adenocarcinoma: 'Krt7', 'Krt20', 'Cdx2',
#Differential diagnosis: 'Muc1', 'Muc2', 'Muc5ac',
#Un-differential cancer(stemness): 'Sox2', 'Sox4', 'Oct4', 'Nanog'.

sc.pl.dotplot(adata, var_names=['Krt7', 'Krt20', 'Cdx2','Muc1', 'Muc2', 'Muc5ac','Sox2', 'Sox4'], groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0, swap_axes=True, dot_min=0, save='tumor_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color=['Krt7', 'Krt20', 'Cdx2','Muc1', 'Muc2', 'Muc5ac','Sox2', 'Sox4'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='WT_tumor_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color=['Krt7', 'Krt20', 'Cdx2','Muc1', 'Muc2', 'Muc5ac','Sox2', 'Sox4'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='KP_tumor_marker')

In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color=['Krt7', 'Krt20', 'Cdx2','Muc1', 'Muc2', 'Muc5ac','Sox2', 'Sox4'],
           legend_loc='right margin', frameon=False, vmin=0,vmax=5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples',save='EKP_tumor_marker')

In [ ]:
sc.pl.dotplot(adata, var_names=['S100a6', 'Tmem176b', 'Cp', 'Eif1', 'Bnip3', 'Pgk1','Prxl2a','Hspb1','Aqp5','Hmgb2','Stmn1','Birc5','Zfp267','Clca1','Gm45864','Gsta1','Mt2','Cbr3','B020031M17Rik','A530040E14Rik','Sp110'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0, save='annotation_top3')

In [ ]:
sc.pl.dotplot(adata, var_names=['Wfdc2','Hp','Cp','Ndufa4','1110008P14Rik','Tmem176a','Gstm1','Tmem176b','Ifitm2','Lcn2','Ppia','Prdx1','Tmsb4x','Cyp2f2','Gsta4','Nme2','Ndufb7','Ndufa1','Gstm2','Atp1b1','Uqcrq','Igfbp7'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0,vmax=4, swap_axes=True, dot_min=0,save='0_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['Eif1','Bnip3','Pgk1','1110038B12Rik','Fau','Epb41l4aos','Ero1l','Lgals3','Car9','Gapdh','Bsg','Tpt1','2410006H16Rik','Btg1','Gpi1','Ccni','Lix1','Txnip','Gabrb1','Cryba4','Gm48604','A2m'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0,vmax=4,swap_axes=True, dot_min=0,save='1_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['Prxl2a','Hspb1','Aqp5','Krt4','Ahnak','Fos','Bcam','Thbs1','Gm1673','Atf3','Abcb1b','Anxa1','Sftpd','Tjp1','Fosb','Egr1','2200002D01Rik','mt-Co3','Zfp36l1','mt-Co1','Klf4','Aspa'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0,save='2_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['Hmgb2','Stmn1','Birc5','Cks1b','Cks2','Pclaf','H2afz','Tuba1b','Cdk1','Top2a','Tubb5','Ube2s','Cdca8','Smc2','Mki67','Ube2c','Dut','Prc1','Cenpf','Cdca3','Cenpa','Ptma'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0,save='3_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['Zfp267','Clca1','Gm45864','Gm35551','Zfp711','Rnf41','Gm20621','Hypm','1700010B13Rik','Usp17lb','Gm5890','Fbxo17','Gspt2','Armh4','Khdc1b','Clps','Gm26707','Sgcd','Hpdl','Gm5157','Gm45659','Gm21718'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0,save='4_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['Gsta1','Mt2','Cbr3','Gpx2','Crip1','Clu','Ptgr1','Vsig2','Lgals4','Fxyd3','Gsta4','Plac8','Esd','AA467197','Mt1','Rpl15','Ociad2','Ugdh','Cbr1','Gm14685','Grik5','Csrp2'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0,save='5_annotation_top22')

In [ ]:
sc.pl.dotplot(adata, var_names=['B020031M17Rik','A530040E14Rik','Sp110','Platr31','Dcc','BC147527','Tbc1d23','Scamp1','AF067061','Ppig','Zfp560','Epm2a','Gm44693','Bola1','Dennd4c','Cwc22','Echdc3','Arhgef26','Fgf1','Scd3','Cab39','Prex2'], groupby='leiden', use_raw=True, var_group_rotation=0,
              vmin=0, vmax=4,swap_axes=True, dot_min=0,save='6_annotation_top22')

In [ ]:
adata.write('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/3in1.h5ad', compression='gzip')

In [ ]:
WT=adata[adata.obs['batch']=='WT']
WT

In [ ]:
WT.write('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/WT_processed.h5ad', compression='gzip')

In [ ]:
KP=adata[adata.obs['batch']=='KP']
KP

In [ ]:
KP.write('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/KP_processed.h5ad', compression='gzip')

In [ ]:
EKP=adata[adata.obs['batch']=='EKP']
EKP

In [ ]:
EKP.write('/Users/gzou/OneDrive - Inside MD Anderson/Gengyi_DGC/DGC_matrix/EKP_processed.h5ad', compression='gzip')

In [ ]:
# CTLA4 and PD-1 and PD-L1
#CD274=PDL1; PDCD1LG2=PDL2, PDCD1=PD1
#‘area’, ‘count’, ‘width’


sc.tl.score_genes(adata, gene_list=['Ctla4','Pdcd1','Pdcd1lg2','Cd274'], ctrl_size=50, gene_pool=None, n_bins=25, score_name='immune_checkpoint_score', random_state=0, copy=False, use_raw=None)
sc.pl.dotplot(adata, var_names='immune_checkpoint_score', groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0,swap_axes=True, dot_min=0)

sc.pl.violin(adata, stripplot=False,keys='immune_checkpoint_score', groupby='batch', use_raw=True)



In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='KP'], color='Tigit', legend_loc='right margin', frameon=False, vmin=0,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color='Tigit', legend_loc='right margin', frameon=False, vmin=0,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')


In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color='Pvr', legend_loc='right margin', frameon=False, vmin=0,vmax=1.5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')

sc.pl.umap(adata[adata.obs['batch']=='KP'], color='Pvr', legend_loc='right margin', frameon=False, vmin=0,vmax=1.5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color='Pvr', legend_loc='right margin', frameon=False, vmin=0,vmax=1.5,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')


In [ ]:
sc.pl.dotplot(adata, var_names='Tigit', groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0,swap_axes=True, dot_min=0)


In [ ]:
sc.pl.dotplot(adata, var_names='Pvr', groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0,swap_axes=True, dot_min=0,save='Pvr')


In [ ]:
sc.pl.umap(adata[adata.obs['batch']=='WT'], color='Nectin2', legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')

sc.pl.umap(adata[adata.obs['batch']=='KP'], color='Nectin2', legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')
sc.pl.umap(adata[adata.obs['batch']=='EKP'], color='Nectin2', legend_loc='right margin', frameon=False, vmin=0,vmax=3,add_outline=True, outline_width=(0.05,0.05), size=50, use_raw=True, cmap='Purples')


In [ ]:
sc.pl.dotplot(adata, var_names='Pilra', groupby='batch', use_raw=True, var_group_rotation=0,
              vmin=0,swap_axes=True, dot_min=0,save='Pilra')
